In [0]:
import numpy as np
import random
import sys
import copy
a = str(sys.argv[0])
if a == "print":
  
class board :

  def __init__(self, num_stones, store1 = 0, store2 = 0):

    self.state = [1 , [store1] + [num_stones]*6 , [num_stones]*6 + [store2]]
    print("       6   5    4     3    2    1")
    print("    ------------------------------")
    print("{:02}  | {:02} | {:02} | {:02} || {:02} | {:02} | {:02}".format(*self.state[1]))
    print("---------------------------------------")
    print("   | {:02} | {:02} | {:02} || {:02} | {:02} | {:02} | {:02}".format(*self.state[2]))
    print("   --------------------------------")
    print("      1    2    3     4    5    6\n")

  def __repr__(self):
    one = "       6   5    4     3    2    1\n"
    two = "    ------------------------------\n"
    three = "{:02}  | {:02} | {:02} | {:02} || {:02} | {:02} | {:02}\n".format(*self.state[1])
    four = "---------------------------------------\n"
    five = "   | {:02} | {:02} | {:02} || {:02} | {:02} | {:02} | {:02}\n".format(*self.state[2])
    six = "   --------------------------------\n"
    seven = "      1    2    3     4    5    6\n"
    return one + two + three + four + five + six + seven


def play_player1(state,action,player = 0) :

  while action > 6 or action < 1 or state[1][7 - action] == 0:
    print("Illegal move , play again : ")
    if player == "Human":
      action = int(input())
    elif player == "Random":
      action = random.randint(1,6)
  stones = state[1][7 - action]   
  state[1][7- action] = 0
  dividend = int(stones/13)
  remainder = stones % 13
  for i in range(7):
    state[1][i] = state[1][i] + dividend
  for i in range(6):
    state[2][i] = state[2][i] + dividend
  if remainder < 7- action + 1 :
    for i in range(remainder):
      state[1][7 - action - 1 -i] = state[1][7 - action - 1 -i] + 1
    player1_last = 7 - action - remainder
  elif remainder < 14 - action:
    for i in range(7 - action):
      state[1][i] = state[1][i] + 1
    for i in range(remainder - (7 - action)):
      state[2][i] = state[2][i] + 1
    player1_last = 9999
  else:
    for i in range(7 - action):
      state[1][i] = state[1][i] + 1
    for i in range(6):
      state[2][i] = state[2][i] + 1
    for i in range(remainder - (7 - action) - 6):
      state[1][6 - i] = state[1][6 - i] + 1
    player1_last = 20 - remainder - action 
  state[0] = set_player_flag(state , player1_last)
  return state

def play_player2(state,action,player = 0) :

  while action > 6 or state[2][action - 1] == 0 :
    print("Illegal Move, play again :")
    if player == "Human":
      action = int(input())
    elif player == "Random" :
      action = random.randint(1,6)
  stones = state[2][action - 1]
  state[2][action - 1] = 0
  dividend = int(stones/13)
  remainder = stones % 13
  for i in range(7):
    state[2][i] = state[2][i] + dividend
  for i in range(6):
    state[1][i+1] = state[1][i+1] + dividend
  if remainder < 8 - action :
    for i in range(remainder):
      state[2][action + i] = state[2][action + i] + 1
    player2_last = action + remainder - 1
  elif remainder < 14 - action:
    for i in range(7 - action):
      state[2][i + action] = state[2][i + action] + 1
    for i in range(remainder - (7 - action)):
      state[1][6-i] = state[1][6-i] + 1
    player2_last = 9999
  else:
    for i in range(7 - action):
      state[2][i + action] = state[2][i + action] + 1
    for i in range(6):
      state[1][i+1] = state[1][i+1] + 1
    for i in range(remainder - (7 - action) - 6):
      state[2][i] = state[2][i] + 1
    player2_last = remainder + action - 14
  state[0] = set_player_flag(state , player2_last)
  return state

def set_player_flag(state , last) :

  if state[0] == 1:
    if last == 0:
      state[0] = 1
    elif last != 9999 and state[1][last] == 1 and state[2][last - 1] != 0:
      jump_stones(state , last)
      state[0] = 2
    else:
      state[0] = 2
  else:
    if last == 6:
      state[0] = 2 
    elif last != 9999 and state[2][last] == 1 and state[1][last + 1] != 0:
      jump_stones(state , last)
      state[0] = 1
    else:
      state[0] = 1

  return state[0]

def jump_stones(state , last) :

  if state[0] == 1:
    state[1][0] = state[1][0] + state[1][last] + state[2][last - 1]
    state[1][last] = 0
    state[2][last - 1] = 0
  else:
    state[2][6] = state[2][6] + state[2][last] + state[1][last +1]
    state[2][last] = 0
    state[1][last +1] = 0

def generate_legal_actions(state):
  if state[0] == 1:
    player1_legal_actions = []
    for i in range(6):
      if state[1][i+1] != 0:
        player1_legal_actions.append(7 - (i+1))
    return player1_legal_actions
  else:
    player2_legal_actions = []
    for i in range(6):
      if state[2][i] != 0:
        player2_legal_actions.append(i+1)
    return player2_legal_actions

def terminal_test(state):
  if sum(state[1][1:]) == 0 or sum(state[2][:-1]) == 0:
    return True
  else:
    return False

def utility_function(state):
  return (sum(state[1]) - sum(state[2]))

def next_states(state):
  original_state = copy.deepcopy(state)
  if state[0] == 1:
    player_actions = generate_legal_actions(state)
    board_states = []
    for i in player_actions:
      board_states.append(play_player1(state , i))
      state = copy.deepcopy(original_state)

  else:
    player_actions = generate_legal_actions(state)
    board_states = []
    for i in player_actions:
      board_states.append(play_player2(state , i))
      state = copy.deepcopy(original_state)

  return [board_states , player_actions]

def print_board(state):
  print("       6   5    4     3    2    1")
  print("    ------------------------------")
  print("{:02}  | {:02} | {:02} | {:02} || {:02} | {:02} | {:02}".format(*state[1]))
  print("---------------------------------------")
  print("   | {:02} | {:02} | {:02} || {:02} | {:02} | {:02} | {:02}".format(*state[2]))
  print("   --------------------------------")
  print("      1    2    3     4    5    6\n")

def minimax_value(state, depth = 7) :
  if terminal_test(state) or depth == 0:
    return utility_function(state)
  else :
    if state[0] == 1:
      maxvalue = -9999
      state_space , action_list = next_states(state)
      for node in state_space :
        value = minimax_value(node , depth - 1)
        maxvalue = max(maxvalue , value)
      return maxvalue

    else :
      minvalue = 9999
      state_space , action_list = next_states(state)     
      for node in state_space :
        value = minimax_value(node , depth - 1)
        minvalue = min(minvalue , value)
      return minvalue

def minimax_move(state) :
  original_state = copy.deepcopy(state)
  state_space , action_list = next_states(state)
  minimax_values = []
  for node in state_space:
    minimax_values.append(minimax_value(node))
  minimax_values = np.asarray(minimax_values)
  best_action = action_list[minimax_values.argmax()]
  return best_action , original_state


def HumanVsHuman():
  print("Hello! Welcome to MANCALA! Please choose the initial number of stones : ")
  stones = int(input())
  game1 = board(stones)
  state = game1.state
  while(1):
    if terminal_test(state):
      if (sum(state[1]) > sum(state[2])):
        print("Player 1 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      elif (sum(state[1]) < sum(state[2])):
        print("Player 2 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      else:
        print("Draw!")
        break
    else:
      if state[0] == 1:
        print("Player 1's turn : ")
        move = int(input())
        state = play_player1(state,move,"Human")
        print_board(state)
      else:
        print("Player 2's turn : ")
        move = int(input())
        state = play_player2(state,move, "Human")
        print_board(state)    

def RandomVsRandom():
  print("Hello! Welcome to MANCALA! Please choose the initial number of stones : ")
  stones = int(input())
  game1 = board(stones)
  state = game1.state
  while(1):
    if terminal_test(state):
      if (sum(state[1]) > sum(state[2])):
        print("Player 1 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      elif (sum(state[1]) < sum(state[2])):
        print("Player 2 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      else:
        print("Draw!")
        break
    else:
      if state[0] == 1:
        print("Player 1's turn : ")
        move = random.randint(1,6)
        state = play_player1(state,move,"Random")
        print_board(state)
      else:
        print("Player 2's turn : ")
        move = random.randint(1,6)
        state = play_player2(state,move, "Random")
        print_board(state)

def MinimaxVsRandom():
  print("Hello! Welcome to MANCALA! Please choose the initial number of stones : ")
  stones = int(input())
  game1 = board(stones)
  state = game1.state
  while(1):
    if terminal_test(state):
      if (sum(state[1]) > sum(state[2])):
        print("Player 1 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      elif (sum(state[1]) < sum(state[2])):
        print("Player 2 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      else:
        print("Draw!")
        break
    else:
      if state[0] == 1:
        print("Player 1's turn : ")
        move , state = minimax_move(state)
        state = play_player1(state,move,"Minimax")
        print_board(state)
      else:
        print("Player 2's turn : ")
        move = random.randint(1,6)
        state = play_player2(state,move, "Random")
        print_board(state)  


def MinimaxVsHuman():
  print("Hello! Welcome to MANCALA! Please choose the initial number of stones : ")
  stones = int(input())
  game1 = board(stones)
  state = game1.state
  while(1):
    if terminal_test(state):
      if (sum(state[1]) > sum(state[2])):
        print("Player 1 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      elif (sum(state[1]) < sum(state[2])):
        print("Player 2 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      else:
        print("Draw!")
        break
    else:
      if state[0] == 1:
        print("Player 1's turn : ")
        move , state = minimax_move(state)
        state = play_player1(state,move,"Minimax")
        print_board(state)
      else:
        print("Player 2's turn : ")
        move = int(input())
        state = play_player2(state,move, "Human")
        print_board(state)

def alphabeta_value(state , alpha = -9999 , beta = 9999 , depth = 7):
  if terminal_test(state) or depth ==0:
    return utility_function(state)
  else:
    maxvalue = -9999
    minvalue = 9999
    if state[0] == 1:
      state_space , action_list = next_states(state)
      for i in state_space :
        value = alphabeta_value(i,alpha,beta , depth - 1)
        maxvalue = max(maxvalue , value)
        alpha = max(maxvalue , alpha)
        if alpha >= beta:
          break
      return maxvalue

    else:
      state_space , action_list = next_states(state)
      for i in state_space :
        value = alphabeta_value(i,alpha,beta, depth - 1)
        minvalue = min(value , minvalue)
        beta = min(beta , minvalue)
        if alpha >= beta:
          break
      return minvalue

def alphabeta_move(state) :
  original_state = copy.deepcopy(state)
  state_space , action_list = next_states(state)
  minimax_values = []
  for node in state_space:
    minimax_values.append(alphabeta_value(node))
  minimax_values = np.asarray(minimax_values)
  best_action = action_list[minimax_values.argmax()]
  return best_action , original_state


def AlphaBetaVsRandom():
  print("Hello! Welcome to MANCALA! Please choose the initial number of stones : ")
  stones = int(input())
  game1 = board(stones)
  state = game1.state
  while(1):
    if terminal_test(state):
      if (sum(state[1]) > sum(state[2])):
        print("Player 1 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      elif (sum(state[1]) < sum(state[2])):
        print("Player 2 Wins!!")
        print("Final score is {} Vs {}".format(sum(state[1]), sum(state[2])))
        break
      else:
        print("Draw!")
        break
    else:
      if state[0] == 1:
        print("Player 1's turn : ")
        move , state = alphabeta_move(state)
        state = play_player1(state,move,"AlphaBeta")
        print_board(state)
      else:
        print("Player 2's turn : ")
        move = random.randint(1,6)
        state = play_player2(state,move, "Random")
        print_board(state)  
    



In [0]:
import time
start_time = time.time()
AlphaBetaVsRandom()
print("--- %s seconds ---" % (time.time() - start_time))

Hello! Welcome to MANCALA! Please choose the initial number of stones : 
4
       6   5    4     3    2    1
    ------------------------------
00  | 04 | 04 | 04 || 04 | 04 | 04
---------------------------------------
   | 04 | 04 | 04 || 04 | 04 | 04 | 00
   --------------------------------
      1    2    3     4    5    6

Player 1's turn : 
       6   5    4     3    2    1
    ------------------------------
01  | 00 | 04 | 04 || 04 | 04 | 04
---------------------------------------
   | 05 | 05 | 05 || 04 | 04 | 04 | 00
   --------------------------------
      1    2    3     4    5    6

Player 2's turn : 
       6   5    4     3    2    1
    ------------------------------
01  | 00 | 04 | 04 || 04 | 05 | 05
---------------------------------------
   | 05 | 05 | 05 || 04 | 00 | 05 | 01
   --------------------------------
      1    2    3     4    5    6

Player 1's turn : 
       6   5    4     3    2    1
    ------------------------------
02  | 01 | 05 | 05 || 05 | 00 | 05
--

In [0]:
import time
start_time = time.time()
MinimaxVsRandom()
print("--- %s seconds ---" % (time.time() - start_time))

Hello! Welcome to MANCALA! Please choose the initial number of stones : 
4
       6   5    4     3    2    1
    ------------------------------
00  | 04 | 04 | 04 || 04 | 04 | 04
---------------------------------------
   | 04 | 04 | 04 || 04 | 04 | 04 | 00
   --------------------------------
      1    2    3     4    5    6

Player 1's turn : 
       6   5    4     3    2    1
    ------------------------------
01  | 00 | 04 | 04 || 04 | 04 | 04
---------------------------------------
   | 05 | 05 | 05 || 04 | 04 | 04 | 00
   --------------------------------
      1    2    3     4    5    6

Player 2's turn : 
       6   5    4     3    2    1
    ------------------------------
01  | 00 | 04 | 04 || 04 | 05 | 05
---------------------------------------
   | 05 | 05 | 05 || 04 | 00 | 05 | 01
   --------------------------------
      1    2    3     4    5    6

Player 1's turn : 
       6   5    4     3    2    1
    ------------------------------
02  | 01 | 05 | 05 || 05 | 00 | 05
--